# TP3 --- RÉSOLUTION DE SYSTÈMES LINÉAIRES

[Python 3 -- testé avec Python 3.8.5]

Le thème de ce TP est la résolution de systèmes d'équations linéaires de la forme $AX = B$ pour une matrice $A \in M_n(\mathbb{R})$ et un vecteur $B \in M_{n,1}(\mathbb{R})$. On implémentera la méthode du pivot de Gauss.

Chargement des paquets utiles :

In [1]:
import numpy as np

## 0. Commandes utiles

Voici quelques commandes utiles dont vous pourrez avoir besoin dans ce TP. La commande `np.array` transforme une liste de listes en une matrice. Ainsi, une matrice $M = (a_{i, j})_{\substack{0 \leqslant i < n \\ 0 \leqslant j < p}} \in \mathcal{M}_{n, p}(\mathbb{R})$ sera représentée par un tableau `M` (de type `ndarray`) grâce à la commande :
```
M = np.array([[a_00, ..., a_0(p-1)], ..., [a_(n-1)0, ..., a_(n-1)(p-1)]], dtype=float)
```

**Attention** : Pour suivre la convention de Python, nous numéroterons les coefficients à partir de $0$.

**Attention** : Pour travailler avec des réels, nous précisons bien l'option `dtype=float`.

Alors :
- `np.shape(M)` renvoie la taille $(n, p)$ de $M$ ;
- `M[i, j]` renvoie le coefficient $a_{i, j}$ ;
- `M[i, :]` renvoie la ligne $i$ de $M$ ;
- `M[:, j]` renvoie la colonne $j$ de $M$ ;
- `M[i1:i2, j1:j2]` renvoie la matrice extraite $(a_{i, j})_{\substack{i_1 \leqslant i < i_2 \\ j_1 \leqslant j < j_2}}$ ;
- `np.concatenate((M, N), axis=0)` renvoie une matrice contenant $M$ et $N$ l'une au-dessus de l'autre.
- `np.concatenate((M, N), axis=1)` renvoie une matrice contenant $M$ et $N$ côte à côte.

Observer et comprendre les commandes ci-dessous.

In [2]:
M = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]], dtype=float)
print("M =\n", M)
print("np.shape(M) =", np.shape(M))
print("M[0, 2] =\n", M[0, 2])
print("M[0, :] =\n", M[0, :])
print("M[:, 1] =\n", M[:, 1])
print("M[1:3, 2:4] =\n", M[1:3, 2:4])

M =
 [[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]]
np.shape(M) = (3, 4)
M[0, 2] =
 3.0
M[0, :] =
 [1. 2. 3. 4.]
M[:, 1] =
 [ 2.  6. 10.]
M[1:3, 2:4] =
 [[ 7.  8.]
 [11. 12.]]


On peut ainsi multiplier une ligne d'une matrice par un nombre, ou remplacer une ligne par elle-même plus (un multiple d')une autre :

In [3]:
M = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]], dtype=float)
print(M)

M[0, :] = 4 * M[0, :]
print(M)

M[1, :] = M[1, :] + M[2, :]
print(M)

M[2, :] = M[2, :] - 2 * M[0, :]
print(M)

[[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]]
[[ 4.  8. 12. 16.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]]
[[ 4.  8. 12. 16.]
 [14. 16. 18. 20.]
 [ 9. 10. 11. 12.]]
[[  4.   8.  12.  16.]
 [ 14.  16.  18.  20.]
 [  1.  -6. -13. -20.]]


**Attention** : Quand on affecte une sous-matrice à une nouvelle matrice, l'espace mémoire n'est pas dupliqué. Pour dupliquer l'expace mémoire il faut créer une nouvelle matrice avec `np.array`. Comparer les deux blocs suivants.

In [4]:
M = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]], dtype=float)

print("M =\n", M)

L_1 = M[1, :]    # L_1 pointe vers la ligne 1 de la matrice M.
L_1[1] = 0

print("L_1 =\n", L_1)
print("M =\n", M)

M =
 [[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]]
L_1 =
 [5. 0. 7. 8.]
M =
 [[ 1.  2.  3.  4.]
 [ 5.  0.  7.  8.]
 [ 9. 10. 11. 12.]]


In [5]:
M = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]], dtype=float)

print("M =\n", M)

L_1 = np.array(M[1, :])    # L_1 est une nouvelle matrice, indépendante de M.
L_1[1] = 0

print("L_1 =\n", L_1)
print("M =\n", M)

M =
 [[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]]
L_1 =
 [5. 0. 7. 8.]
M =
 [[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]]


Enfin, pour concaténer deux matrices, on utilise la commande `np.concatenate`. Observer et comprendre les commandes ci-dessous.

In [6]:
M = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]], dtype=float)
print("M =\n", M)
N = np.array([[13, 14, 15, 16], [17, 18, 19, 20], [21, 22, 23, 24]], dtype=float)
print("N =\n", N)

print("np.concatenate((M, N), axis=0) =\n", np.concatenate((M, N), axis=0))
print("np.concatenate((M, N), axis=1) =\n", np.concatenate((M, N), axis=1))

M =
 [[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]]
N =
 [[13. 14. 15. 16.]
 [17. 18. 19. 20.]
 [21. 22. 23. 24.]]
np.concatenate((M, N), axis=0) =
 [[ 1.  2.  3.  4.]
 [ 5.  6.  7.  8.]
 [ 9. 10. 11. 12.]
 [13. 14. 15. 16.]
 [17. 18. 19. 20.]
 [21. 22. 23. 24.]]
np.concatenate((M, N), axis=1) =
 [[ 1.  2.  3.  4. 13. 14. 15. 16.]
 [ 5.  6.  7.  8. 17. 18. 19. 20.]
 [ 9. 10. 11. 12. 21. 22. 23. 24.]]


## 1. Un premier exemple

Le système linéaire suivant :
$$
(\mathcal{S}) \quad \left\lbrace
\begin{array}{rcrcrlc}
2 x_0 & + & 4 x_1 & - & x_2 & = & 1 \\
4 x_0 & & & - & 2 x_2 & = & 0 \\
x_0 & + & 2 x_1 & + & x_2 & = & 2
\end{array}
\right.
$$
est équivalent à l'équation matricielle $A X = B$ d'inconnue $X = \begin{pmatrix} x_0 \\ x_1 \\ x_2 \end{pmatrix}$, où $A$ est la matrice du système et $B$ le vecteur colonne des seconds membres. Définir la matrice `A` et le vecteur `B` correspondants.

In [7]:
# Solution
A = np.array([[2,4,-1],[4,0,-2],[1,2,1]], dtype=float)
B = np.array([[1],[0],[2]], dtype=float)
# Fin solution

On souhaite résoudre le système $(\mathcal{S})$ en utilisant la méthode du pivot de Gauss. Les seules opérations élémentaires autorisées sont :
- les **permutations** de lignes $L_i \leftrightarrow L_j$ ($i \neq j$) ;
- les **dilatations** de lignes $L_i \leftarrow \alpha L_i$ ($\alpha \in \mathbb{R} \setminus \{ 0 \}$) ;
- et les **transvections** sur les lignes $L_i \leftarrow L_i + \beta L_j$ ($i \neq j$ et $\beta \in \mathbb{R}$).

On définit, à l'aide de la commande `np.concatenate`, la matrice `S` (élément de $\mathcal{M}_{3,4}(\mathbb{R})$) dont les 3 premières colonnes correspondent à la matrice `A`, et la dernière colonne au vecteur `B`. Cette matrice `S` représente complètement notre système linéaire.

In [8]:
S = np.concatenate((A, B), axis=1)
print("S =\n", S)

S =
 [[ 2.  4. -1.  1.]
 [ 4.  0. -2.  0.]
 [ 1.  2.  1.  2.]]


Effectuer sur la matrice `S` les opérations élémentaires correspondant à l'algorithme du pivot de Gauss et qui aboutissent à une matrice de la forme
$$
S_{\text{échelonné}} =
\begin{pmatrix}
1 & * & * & b'_0 \\
0 & 1 & * & b'_1 \\
0 & 0 & 1 & b'_2
\end{pmatrix} .
$$

In [9]:
S_echelonne = np.array(S)    # On copie la matrice S pour que la matrice S elle-même ne soit pas modifiée.

# Solution
for k in range(4):
    S_echelonne[1,k] = S_echelonne[1,k] - 2* S_echelonne[0,k]
    S_echelonne[2,k] = S_echelonne[2,k] - (1/2)*S_echelonne[0,k]
    S_echelonne[0,k] = (1/2)*S_echelonne[0,k]
    S_echelonne[1,k] = (1/-8)* S_echelonne[1,k] 
    S_echelonne[2,k] = S_echelonne[2,k]*(2/3)
     
print("S =\n", S_echelonne)
# Fin solution

S =
 [[ 1.    2.   -0.5   0.5 ]
 [-0.    1.   -0.    0.25]
 [ 0.    0.    1.    1.  ]]


Enfin, on va effectuer sur la matrice `S_echelonne` des opérations élémentaires pour aboutir à une matrice de la forme
$$
S_{\text{résolu}} =
\begin{pmatrix}
1 & 0 & 0 & b''_0 \\
0 & 1 & 0 & b''_1 \\
0 & 0 & 1 & b''_2
\end{pmatrix} .
$$

Effectuer une suite de transvections sur les lignes pour transformer la colonne 2 de $S_{\text{échelonné}}$ en la colonne $\begin{pmatrix} 0 \\ 0 \\ 1 \end{pmatrix}$.

In [10]:
S_resolu = np.array(S_echelonne)    # On copie la matrice S_echelonne et on travaillera sur S_resolu.

# Solution
for k in range(4):
    S_resolu[0,k] = S_resolu[0,k] + 0.5* S_resolu[2,k]
print("S =\n", S_resolu)
# Fin solution

S =
 [[ 1.    2.    0.    1.  ]
 [-0.    1.   -0.    0.25]
 [ 0.    0.    1.    1.  ]]


Effectuer une transvection sur les lignes pour transformer la colonne 1 en la colonne $\begin{pmatrix} 0 \\ 1 \\ 0 \end{pmatrix}$ (sans changer la dernière colonne).

In [11]:
# Solution
for k in range(4):
    S_resolu[0,k] = S_resolu[0,k] - 2 * S_resolu[1,k]
print("S =\n", S_resolu)
# Fin solution

S =
 [[ 1.    0.    0.    0.5 ]
 [-0.    1.   -0.    0.25]
 [ 0.    0.    1.    1.  ]]


La matrice obtenue correspond au système résolu
$$
(\mathcal{S}_\text{résolu}) \quad \left\lbrace
\begin{array}{rcrcrl}
x_0 & & & & & = b''_0 \\
& & x_1 & & & = b''_1 \\
& & & & x_2 & = b''_2 .
\end{array}
\right.
$$
On peut vérifier le résultat obtenu à l'aide de la fonction `np.linalg.solve` :

In [12]:
np.linalg.solve(A, B)

array([[0.5 ],
       [0.25],
       [1.  ]])

## 2. Opérations élémentaires

On définit la fonction `Permutation(M, i, j)` qui renvoie la matrice obtenue à partir d'une matrice `M` en échangeant les lignes `i` et `j`.

In [13]:
def Permutation(M, i, j):
    M_1 = np.array(M)    # On copie la matrice M pour que la matrice M elle-même ne soit pas modifiée.
    M_1[i, :] = M[j, :]
    M_1[j, :] = M[i, :]
    return(M_1)

Définir de même des fonctions `Dilatation(M, i, alpha)` et `Transvection(M, i, j, beta)` qui prennent en argument une matrice `M` et les paramètres d'une opération élémentaire sur les lignes (définie dans la section précédente), et renvoient la matrice modifiée en conséquence.

In [41]:
def Dilatation(M, i, beta):
    # Solution
    
    N = np.array(M)
    N[i,:] = beta * M[i,:]
    return(N)

    # Fin solution

def Transvection(M, i, j, beta):
    # Solution
    N = np.array(M)
    N[i,:] = N[i,:] + beta * M[j,:]
    return(N)
   
    # Fin solution

Tester les fonctions `Permutation(M, i, j)`, `Dilatation(M, i, alpha)` et `Transvection(M, i, j, beta)` sur la matrice
$$
C = \begin{pmatrix}
1 & 1 & -1 \\
1 & 0 & 0 \\
2 & 0 & 1 \\
3 & -2 & 0
\end{pmatrix} .
$$

In [44]:
C = np.array([[1, 1, -1], [1, 0, 0], [2, 0, 1], [3, -2, 0]], dtype=float)
print("C =\n", C)

# Solution
print("Dilatation(C,0,2)\n", Dilatation(C,0,2))
print("Permutation(C,0,3) =\n", Permutation(C,0,3))
print("Transvection(C,1,2) =\n", Transvection(C,1,2,2))
# Fin solution

C =
 [[ 1.  1. -1.]
 [ 1.  0.  0.]
 [ 2.  0.  1.]
 [ 3. -2.  0.]]
Dilatation(C,0,2)
 [[ 2.  2. -2.]
 [ 1.  0.  0.]
 [ 2.  0.  1.]
 [ 3. -2.  0.]]
Permutation(C,0,3) =
 [[ 3. -2.  0.]
 [ 1.  0.  0.]
 [ 2.  0.  1.]
 [ 1.  1. -1.]]
Transvection(C,1,2) =
 [[ 1.  1. -1.]
 [ 5.  0.  2.]
 [ 2.  0.  1.]
 [ 3. -2.  0.]]


Écrire une fonction `SuiteTransvections(M, beta)` qui prend en argument une matrice `M` à $n$ lignes et une liste `beta` contenant $n - 1$ scalaires $\beta_1$, ..., $\beta_{n-1}$, et qui renvoie la matrice obtenue en transformant $M$ par la suite d'opérations :
\begin{align*}
L_1 & \leftarrow L_1 + \beta_1 L_0 , \\
L_2 & \leftarrow L_2 + \beta_2 L_0 , \\
& \vdots \\
L_{n-1} & \leftarrow L_{n-1} + \beta_{n-1} L_0 .
\end{align*}

In [37]:
def SuiteTransvections(M, beta):
    (n, p) = np.shape(M)    # On récupère ainsi la taille (n, p) de la matrice M.
    M_modifiee = np.array(M)    # On copie la matrice M pour que la matrice M elle-même ne soit pas modifiée.
    # Solution
    
    # Fin solution

Tester la fonction `SuiteTransvections(M, beta)` sur la matrice $C$.

In [ ]:
SuiteTransvections(C, [-1, -2, -3])

## 3. Résolution d'un système triangulaire inversible

L'algorithme du pivot de Gauss permet, en utilisant uniquement des opérations élémentaires sur les lignes, de transformer un système linéaire quelconque $A X = B$ en un système **échelonné**, c'est-à-dire un système qui s'écrit sous la forme $E X = \hat{B}$, où la matrice $E$ est **échelonnée**, au sens où :
* le premier coefficient non nul de chaque ligne est égal à $1$ ;
* quand on passe d'une ligne à la suivante, le nombre de zéros en début de ligne augmente strictement, jusqu'à rester (éventuellement) constamment égal au nombre de colonnes de $E$.

Une fois le système mis sous cette forme, il devient facile à résoudre, en utilisant seulement des transvections sur les lignes.

Si l'on part d'une matrice $A$ carrée et inversible, on aboutit à une matrice $E$ triangulaire supérieure dont tous les éléments diagonaux sont égaux à $1$ :
$$
E = \begin{pmatrix}
1 & & (e_{ij}) \\
& \ddots & \\
(0) & & 1
\end{pmatrix}
$$

Reprendre la matrice $S_{\text{échelonné}}$ (obtenue dans la section 1) et utiliser la fonction `Transvection(M, i, j, beta)` pour obtenir la matrice $S_{\text{résolu}}$.

In [ ]:
# Solution

# Fin solution

Écrire une fonction `ResolutionEchelonne(S_echelonne)` qui prend en argument une matrice `S_echelonne` à $n$ lignes, dont les $n$ premières colonnes forment une matrice $E$ triangulaire supérieure avec des $1$ sur la diagonale, et qui renvoie la matrice modifiée par la suite de transvections permettant de transformer $E$ en la matrice identité.

In [ ]:
def ResolutionEchelonne(S_echelonne):
    # Solution
    
    # Fin solution

Tester la fonction `ResolutionEchelonne(S_echelonne)` en résolvant les systèmes
$$
(\mathcal{S}_0) \quad \left\lbrace
\begin{array}{rcrcrlc}
x_0 & + & x_1 &- &2 x_2 & =& 0 \\
& & x_1 & + & 3 x_2 & = & -1 \\
& & & & x_2 & = & 1
\end{array}
\right.
$$
et
$$
(\mathcal{S}_1) \quad \left\lbrace
\begin{array}{rcrcrcrlc}
x_0 & - & 2 x_1& + & x_2 & + & 5 x_3 & = & 2 \\
& & x_1 & + & 3 x_2 & - & x_3 & = & 0 \\
& & & & x_2 & + & 4 x_3 & = & 1 \\
& & & & & & x_3 & = & -1 .
\end{array}
\right.
$$

In [ ]:
E_0 = np.array([[1, 1, -2], [0, 1, 3], [0, 0, 1]], dtype=float)
B_0 = np.array([[0], [-1], [1]], dtype=float)
S_0 = np.concatenate((E_0, B_0), axis=1)

print("S_0 =\n", S_0)
print("S_0_resolu :\n", ResolutionEchelonne(S_0))
print("Vérification : np.linalg.solve(E_0, B_0) =\n", np.linalg.solve(E_0, B_0))

S_1 = np.array([[1, -2, 1, 5, 2], [0, 1, 3, -1, 0], [0, 0, 1, 4, 1], [0, 0, 0, 1, -1]], dtype=float)

print("S_1 =\n", S_1)
print("S_1_resolu :\n", ResolutionEchelonne(S_1))
print("Vérification : np.linalg.solve(S_1[:, 0:4], S_1[:, 4]) =\n", np.linalg.solve(S_1[:, 0:4], S_1[:, 4]))

## 4. Algorithme du pivot de Gauss

### 4.1. Choix d'un pivot

Définir une fonction `ChoixPivot(S)` qui prend en argument une matrice `S` de taille quelconque et renvoie une matrice `S_choix` obtenue à partir de `S` en trouvant dans la colonne $0$ de `S` un élément non nul (pivot), puis en permutant et dilatant des lignes de sorte que le pivot soit en ligne $0$ et égal à $1$. Si ce n'est pas possible, la fonction affiche un message d'erreur et renvoie `None`.

Pour trouver un élément non nul, on choisira l'élément de valeur absolue maximale en utilisant les fonctions `abs` et `np.argmax`.

In [ ]:
def ChoixPivot(S):
    i = np.argmax(abs(S[:, 0]))
    # Solution
    
    # Fin solution

Tester la fonction `ChoixPivot(S)` sur les matrices suivantes : `S_2 = S` (matrice définie dans la section 1),
$$
S_3 = \begin{pmatrix}
0 & 1 & 1 & 0 \\
1 & 0 & 0 & 1 \\
2 & 0 & 3 & 0
\end{pmatrix} ,
\quad
S_4 = \begin{pmatrix}
0 & 1 & 0 \\
0 & 0 & 2
\end{pmatrix}
$$
et `S_5 = np.random.randn(10, 11)` (matrice aléatoire).

In [ ]:
# S_2
S_2 = S    # Matrice définie dans la section 1.
print("S_2 =\n", S_2)

S_2_choix = ChoixPivot(S_2)
print("S_2_choix =\n", S_2_choix)

# S_3
S_3 = np.array([[0, 1, 1, 0], [1, 0, 0, 1], [2, 0, 3, 0]], dtype=float)
print("S_3 =\n", S_3)

S_3_choix = ChoixPivot(S_3)
print("S_3_choix =\n", S_3_choix)

# S_4
S_4 = np.array([[0, 1, 0], [0, 0, 2]], dtype=float)
print("S_4 =\n", S_4)

S_4_choix = ChoixPivot(S_4)
print("S_4_choix =\n", S_4_choix)

# S_5
S_5 = np.random.randn(10, 11)
print("S_5 =\n", np.round(S_5, 2))    # On arrondit les valeurs pour un affichage plus lisible.

S_5_choix = ChoixPivot(S_5)
print("S_5_choix =\n", np.round(S_5_choix, 2))

### 4.2. Élimination à l'aide du pivot

On donne la fonction `EliminationPivot(S_choix)` qui prend en argument une matrice `S_choix` de taille quelconque dont le coefficient $(0, 0)$ est égal à $1$ (autrement dit une sortie de la fonction `ChoixPivot`), qui effectue les transvections nécessaires pour annuler tous les autres coefficients de la colonne $0$, et qui renvoie la matrice ainsi obtenue.

In [ ]:
def EliminationPivot(S_choix):
    beta = -S_choix[1:, 0]
    return SuiteTransvections(S_choix, beta)

Tester la fonction `EliminationPivot(S_choix)` sur les matrices `S_2_choix`, `S_3_choix` et `S_5_choix` ci-dessus.

In [ ]:
print("S_2_choix =\n", S_2_choix)
S_2_elimination = EliminationPivot(S_2_choix)
print("S_2_elimination =\n", S_2_elimination)

print("S_3_choix =\n", S_3_choix)
S_3_elimination = EliminationPivot(S_3_choix)
print("S_3_elimination =\n", S_3_elimination)

print("S_5_choix =\n", np.round(S_5_choix, 2))
S_5_elimination = EliminationPivot(S_5_choix)
print("S_5_elimination =\n", np.round(S_5_elimination, 2))

### 4.3. Échelonnement d'un système inversible

Définir de manière récursive une fonction `Echelonnement(S)` qui prend en argument une matrice `S` dans $\mathcal{M}_{n, n+1}(\mathbb{R})$ correspondant à un système inversible (i.e. les $n$ premières colonnes forment une matrice carrée inversible) et renvoie une matrice `S_triangularise` correspondant à un système triangulaire supérieur avec des 1 sur la diagonale, obtenue à partir de `S` par une suite d'opérations élémentaires. Si le système n'est pas inversible, la fonction renvoie un message d'erreur.

In [ ]:
def Echelonnement(S):
    # Solution
    
    # Fin solution

Tester la fonction `Echelonnement(S)` sur les matrices `S_2`, `S_3` et `S_5` ci-dessus.

In [ ]:
print("S_2 =\n", S_2)
print("S_2_triangularise =\n", Echelonnement(S_2))

print("S_3 =\n", S_3)
print("S_3_triangularise =\n", Echelonnement(S_3))

print("S_5 =\n", np.round(S_5, 2))
print("S_5_triangularise =\n", np.round(Echelonnement(S_5), 2))

## 5. Résolution d'un système inversible et calcul d'inverse

On donne la fonction `Resolution(S)` qui prend en argument une matrice `S` correspondant à un système inversible, et qui renvoie la matrice correspondant au système résolu, dans laquelle la solution se lit sur la dernière colonne.

In [ ]:
def Resolution(S):
    return ResolutionEchelonne(Echelonnement(S))

Tester la fonction `Resolution(S)` avec le système $(\mathcal{S})$ de la section 1.

In [ ]:
print("S =\n", S)
print("S_resolu =\n", Resolution(S))
print("Vérification : np.linalg.solve(A, B) =\n", np.linalg.solve(A, B))

Comment utiliser cette fonction pour calculer l'inverse d'une matrice inversible, par exemple
$$
A = \begin{pmatrix}
2 & 4 & -1 \\
4 & 0 & -2 \\
1 & 2 & 1
\end{pmatrix}\ ?
$$
Comparer le résultat avec la fonction `np.linalg.inv`.

In [ ]:
# Solution

# Fin solution

## 6. Retour sur le choix du pivot (stabilité numérique)

Écrire une fonction `ChoixPivot_2(S)` qui fait la même chose que la fonction `ChoixPivot(S)`, mais en choisissant comme pivot le premier élément non nul de la colonne 0. Écrire également la fonction `Triangularisation_2(S)` correspondante. On en déduit la fonction et `Resolution_2(S)`.

In [ ]:
def ChoixPivot_2(S):
    # Solution
    
    # Fin solution


def Echelonnement_2(S):
    # Solution
    
    # Fin solution


def Resolution_2(S):
    return ResolutionEchelonne(Echelonnement_2(S))

Tester la fonction `Resolution_2(S)` sur le système
$$
(\mathcal{S}_6) \quad \left\lbrace
\begin{array}{rcrcrlc}
0.00000000000001 x & + & y & + & z & = & 1 \\
x & + & y & & & = & 1 \\
2 x & + & y & & & = & 3 .
\end{array}
\right.
$$
et comparer avec `Resolution(S)`.

In [ ]:
S_6 = np.array([[1e-14, 1, 1, 1], [1, 1, 0, 1], [2, 1, 0, 3]], dtype=float)

print("S_6 =\n", S_6)
print("Resolution_2(S_6) =\n", Resolution_2(S_6))
print("Resolution(S_6) =\n", Resolution(S_6))

# Solution

# Fin solution

## 7. Le pivot de Gauss dans le cas général

[*Cette question est difficile et facultative*]

Écrire une fonction qui applique l'algorithme du pivot de Gauss pour donner toutes les solutions d'un système d'équations linéaires quelconque (pas nécessairement carré, ni inversible).